<h1>Datascience Capstone Project</h1>

In this capstone project, hopefully I will learn new skill related location data and how to use it develop cool applciations.

<h1>Assignment - Part 1</h1>

In [2]:
#import numpy and panda
import numpy as np 
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# show the dataframe
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Get coordination for Shah Alam Neighborhood/Section

In [6]:
# city of interest
city = 'Shah Alam, MY'
# number of neighborhoods
section = 36

# get the location of each neighborhoods
for x in range(1, section):
    address = 'Section ' + str(x) + ', ' + city
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address) 
    if (location is not None):
       neighborhood_sec = 'Section ' + str(x)
       neighborhood_lat = location.latitude
       neighborhood_lon = location.longitude
       neighborhoods = neighborhoods.append({'Borough': city,
                                          'Neighborhood': neighborhood_sec,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    else:
       print('No coordinate of {}.'.format(address))  # in case geolocator cannot find the location

No coordinate of Section 7, Shah Alam, MY.
No coordinate of Section 26, Shah Alam, MY.
No coordinate of Section 27, Shah Alam, MY.
No coordinate of Section 28, Shah Alam, MY.
No coordinate of Section 29, Shah Alam, MY.
No coordinate of Section 30, Shah Alam, MY.
No coordinate of Section 31, Shah Alam, MY.
No coordinate of Section 32, Shah Alam, MY.
No coordinate of Section 33, Shah Alam, MY.
No coordinate of Section 34, Shah Alam, MY.


<h3>Render Shah Alam city <h3>

In [96]:
#Shah Alam, Malaysia Lat & Long
city_address = 'Shah Alam, MY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city_address)    
latitude = (location.latitude - 0.1) # but the given coordinate is a bit off
longitude = location.longitude

print('The geograpical coordinate of Shah Alam, are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Shah Alam, are 2.9898294, 101.53116264843747.


In [97]:
# create map of Shah Alam using latitude and longitude values
map_ShahAlam = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ShahAlam)  
    
map_ShahAlam

let's simplify the above map and segment and cluster only the neighborhoods in Shah Alam city, Malaysia. So let's slice the original dataframe and create a new dataframe of the Shah Alam data

In [13]:
#shahalam_data 
ShahAlam_data = neighborhoods
ShahAlam_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,"Shah Alam, MY",Section 1,3.069430,101.500523
1,"Shah Alam, MY",Section 2,3.067846,101.511315
2,"Shah Alam, MY",Section 3,3.075397,101.507429
3,"Shah Alam, MY",Section 4,3.078586,101.511890
4,"Shah Alam, MY",Section 5,3.083228,101.516758


In [14]:
ShahAlam_data.shape

(25, 4)

Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK' # your Foursquare ID
CLIENT_SECRET = 'U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK
CLIENT_SECRET:U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2


In [16]:
ShahAlam_data.loc[3, 'Neighborhood']

'Section 4'

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = ShahAlam_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ShahAlam_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = ShahAlam_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Section 4 are 3.0785863, 101.5118898.


In [22]:
import requests # library to handle requests

Create the GET request URL. Name your URL url.

In [57]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK&client_secret=U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2&v=20180605&ll=3.0785863,101.5118898&radius=1000&limit=100'

Send the GET request and examine the resutls

In [58]:
results = requests.get(url).json()
#results

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Taman Bukit S.U.K,Park,3.080670,101.513317
1,Spiced Pumpkin,Eastern European Restaurant,3.074995,101.509666
2,McCafé,Café,3.074360,101.510338
3,Taman Tasik Shah Alam,Park,3.073930,101.515707
4,Serai Thai Restaurant,Thai Restaurant,3.075783,101.509335


In [60]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

91 venues were returned by Foursquare.


In [27]:
#function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create a new dataframe called scarborough_venues for each neighborhood

In [61]:
ShahAlam_venues = getNearbyVenues(names=ShahAlam_data['Neighborhood'],
                                   latitudes=ShahAlam_data['Latitude'],
                                   longitudes=ShahAlam_data['Longitude']
                                  )

Section 1
Section 2
Section 3
Section 4
Section 5
Section 6
Section 8
Section 9
Section 10
Section 11
Section 12
Section 13
Section 14
Section 15
Section 16
Section 17
Section 18
Section 19
Section 20
Section 21
Section 22
Section 23
Section 24
Section 25
Section 35


In [62]:
print(ShahAlam_venues.shape)
ShahAlam_venues.head()

(505, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Section 1,3.069430,101.500523,Padang Sintetik UiTM Shah Alam,3.065586,101.499007,Soccer Field
1,Section 1,3.069430,101.500523,Kolam Renang UiTM,3.069100,101.500448,Pool
2,Section 1,3.069430,101.500523,Pusat Sukan UiTM,3.067421,101.497327,College Stadium
3,Section 1,3.069430,101.500523,UITM Shah Alam Concert Hall,3.065065,101.499531,Concert Hall
4,Section 2,3.067846,101.511315,yogaonethatIwant2studios,3.069595,101.511850,Athletics & Sports


Let's check how many venues were returned for each neighborhood

In [63]:
ShahAlam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Section 1,4,4,4,4,4,4
Section 10,69,69,69,69,69,69
Section 11,17,17,17,17,17,17
Section 12,20,20,20,20,20,20
Section 13,34,34,34,34,34,34
Section 14,18,18,18,18,18,18
Section 15,16,16,16,16,16,16
Section 16,18,18,18,18,18,18
Section 17,26,26,26,26,26,26


Let's find out how many unique categories can be curated from all the returned venues

In [64]:
print('There are {} uniques categories.'.format(len(ShahAlam_venues['Venue Category'].unique())))

There are 120 uniques categories.


<h3>Analyze each neighborrhoods<h3>

In [65]:
# one hot encoding
ShahAlam_onehot = pd.get_dummies(ShahAlam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ShahAlam_onehot['Neighborhood'] = ShahAlam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ShahAlam_onehot.columns[-1]] + list(ShahAlam_onehot.columns[:-1])
ShahAlam_onehot = ShahAlam_onehot[fixed_columns]

ShahAlam_onehot.head()

,Neighborhood,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,...,Street Food Gathering,Supermarket,Tailor Shop,Thai Restaurant,Theater,Thrift / Vintage Store,Track Stadium,Train Station,Volleyball Court,Water Park
0,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Section 2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
ShahAlam_onehot.shape

(505, 121)

In [67]:
ShahAlam_grouped = ShahAlam_onehot.groupby('Neighborhood').mean().reset_index()
ShahAlam_grouped

,Neighborhood,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,...,Street Food Gathering,Supermarket,Tailor Shop,Thai Restaurant,Theater,Thrift / Vintage Store,Track Stadium,Train Station,Volleyball Court,Water Park
0,Section 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Section 10,0.014493,0.014493,0.000000,0.072464,0.000000,0.014493,0.000000,0.043478,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493,0.000000
2,Section 11,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000
3,Section 12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.05,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Section 13,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.029412,0.029412,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000
5,Section 14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Section 15,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Section 16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Section 17,0.038462,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.038462,0.00,0.000000,0.076923,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000
9,Section 18,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [68]:
ShahAlam_grouped.shape

(25, 121)

Print each neighborhood along with the top 5 most common venues

In [69]:
num_top_venues = 5

for hood in ShahAlam_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ShahAlam_grouped[ShahAlam_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Section 1----
                       venue  freq
0            College Stadium  0.25
1                       Pool  0.25
2               Soccer Field  0.25
3               Concert Hall  0.25
4  Indian Chinese Restaurant  0.00


----Section 10----
                 venue  freq
0     Malay Restaurant  0.12
1                 Café  0.09
2     Asian Restaurant  0.07
3  Japanese Restaurant  0.04
4               Bakery  0.04


----Section 11----
              venue  freq
0  Malay Restaurant  0.18
1        Food Truck  0.12
2  Asian Restaurant  0.12
3        Food Court  0.12
4  Halal Restaurant  0.06


----Section 12----
               venue  freq
0               Pool  0.10
1   Malay Restaurant  0.10
2         Food Court  0.10
3  Convenience Store  0.05
4  Electronics Store  0.05


----Section 13----
               venue  freq
0   Malay Restaurant  0.21
1               Café  0.09
2  Convenience Store  0.09
3        Pizza Place  0.09
4         Skate Park  0.06


----Section 14----
             

In [70]:
# sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ShahAlam_grouped['Neighborhood']

for ind in np.arange(ShahAlam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ShahAlam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Section 1,College Stadium,Soccer Field,Pool,Concert Hall,Comfort Food Restaurant,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop
1,Section 10,Malay Restaurant,Café,Asian Restaurant,Bakery,Japanese Restaurant,Indian Restaurant,Juice Bar,Spa,Ice Cream Shop,Coffee Shop
2,Section 11,Malay Restaurant,Asian Restaurant,Food Court,Food Truck,Building,Park,Electronics Store,Restaurant,Café,Halal Restaurant
3,Section 12,Pool,Malay Restaurant,Food Court,Coffee Shop,Spa,Music Store,Moving Target,Convenience Store,Electronics Store,Karaoke Bar
4,Section 13,Malay Restaurant,Convenience Store,Pizza Place,Café,Gym,Fast Food Restaurant,Skate Park,Restaurant,Bakery,Bar


<h3>Cluster Neighborhoods<h3>

In [72]:
# set number of clusters
kclusters = 7

ShahAlam_grouped_clustering = ShahAlam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ShahAlam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 4, 4, 4, 4, 4, 1, 4, 4, 4], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ShahAlam_merged = ShahAlam_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ShahAlam_merged = ShahAlam_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [55]:
ShahAlam_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Shah Alam, MY",Section 1,3.069430,101.500523,1,College Stadium,Soccer Field,Pool,Concert Hall,Comfort Food Restaurant,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop
1,"Shah Alam, MY",Section 2,3.067846,101.511315,4,Café,Water Park,College Academic Building,Athletics & Sports,Print Shop,Beach,Street Art,Farmers Market,Exhibit,Electronics Store
2,"Shah Alam, MY",Section 3,3.075397,101.507429,4,Fast Food Restaurant,Ice Cream Shop,Asian Restaurant,Burger Joint,Malay Restaurant,Convenience Store,Arts & Crafts Store,Food Truck,Indian Restaurant,Café
3,"Shah Alam, MY",Section 4,3.078586,101.511890,0,Fast Food Restaurant,Gym / Fitness Center,Malay Restaurant,Asian Restaurant,Indian Restaurant,Building,Park,Business Service,Eastern European Restaurant,Food Truck
4,"Shah Alam, MY",Section 5,3.083228,101.516758,4,Soccer Stadium,Soccer Field,Food Truck,Café,Seafood Restaurant,Park,Garden,Golf Course,College Stadium,Comfort Food Restaurant


In [74]:
import math

 let's visualize the resulting clusters

In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]  

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ShahAlam_merged['Latitude'], ShahAlam_merged['Longitude'], ShahAlam_merged['Neighborhood'], ShahAlam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if not(math.isnan(cluster)):
        folium.CircleMarker(
           [lat, lon],
           radius=5,
           popup=label,
           color=rainbow[int(cluster-1)],
           fill=True,
           fill_color=rainbow[int(cluster-1)],
           fill_opacity=0.7).add_to(map_clusters)      
map_clusters

<h3> Examine the clusters<h3>

Cluster 1

In [100]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 0, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Section 19,Soccer Field,Boutique,Dessert Shop,Indian Chinese Restaurant,Electronics Store,Field,Fast Food Restaurant,Farmers Market,Exhibit,Water Park


cluster 2

In [101]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 1, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Section 6,Asian Restaurant,Malay Restaurant,Burger Joint,Gym / Fitness Center,Fish Market,Convenience Store,Café,Cafeteria,Martial Arts Dojo,Farmers Market
13,Section 15,Asian Restaurant,Malay Restaurant,Boutique,Burger Joint,Food,Convenience Store,Breakfast Spot,Thai Restaurant,Chinese Restaurant,Pool Hall
18,Section 20,Malay Restaurant,Asian Restaurant,Pizza Place,Restaurant,Convenience Store,Thai Restaurant,Café,Exhibit,Electronics Store,Eastern European Restaurant
20,Section 22,Food Court,Malay Restaurant,Breakfast Spot,Water Park,Field,Concert Hall,Convenience Store,Department Store,Dessert Shop,Diner


cluster 3

In [102]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 2, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Section 21,Cemetery,Campground,Water Park,Field,Concert Hall,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop


cluster 4

In [103]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 3, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Section 35,Art Gallery,Lake,Basketball Court,BBQ Joint,Exhibit,Field,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop


cluster 5

In [104]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 4, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Section 3,Fast Food Restaurant,Ice Cream Shop,Asian Restaurant,Burger Joint,Malay Restaurant,Convenience Store,Arts & Crafts Store,Food Truck,Indian Restaurant,Café
3,Section 4,Fast Food Restaurant,Gym / Fitness Center,Malay Restaurant,Asian Restaurant,Indian Restaurant,Building,Park,Business Service,Eastern European Restaurant,Food Truck
4,Section 5,Soccer Stadium,Soccer Field,Food Truck,Café,Seafood Restaurant,Park,Garden,Golf Course,College Stadium,Comfort Food Restaurant
6,Section 8,Convenience Store,Malay Restaurant,Burger Joint,Pizza Place,Basketball Court,Playground,Clothing Store,Park,Indian Restaurant,Breakfast Spot
7,Section 9,Café,Asian Restaurant,Malay Restaurant,Indian Restaurant,Bakery,Japanese Restaurant,Restaurant,Ice Cream Shop,Coffee Shop,Farmers Market
8,Section 10,Malay Restaurant,Café,Asian Restaurant,Bakery,Japanese Restaurant,Indian Restaurant,Juice Bar,Spa,Ice Cream Shop,Coffee Shop
9,Section 11,Malay Restaurant,Asian Restaurant,Food Court,Food Truck,Building,Park,Electronics Store,Restaurant,Café,Halal Restaurant
10,Section 12,Pool,Malay Restaurant,Food Court,Coffee Shop,Spa,Music Store,Moving Target,Convenience Store,Electronics Store,Karaoke Bar
11,Section 13,Malay Restaurant,Convenience Store,Pizza Place,Café,Gym,Fast Food Restaurant,Skate Park,Restaurant,Bakery,Bar
12,Section 14,Halal Restaurant,Café,Malay Restaurant,Steakhouse,Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Indian Restaurant,Sporting Goods Shop


In [105]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 5, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Section 1,College Stadium,Soccer Field,Pool,Concert Hall,Comfort Food Restaurant,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop


In [106]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 6, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Section 2,Café,Water Park,College Academic Building,Athletics & Sports,Print Shop,Beach,Street Art,Farmers Market,Exhibit,Electronics Store
